In [0]:
# 99_utils/99_setup_kardia_autoloader_env.ipynb
# One-time setup for a new Databricks workspace.
# Creates raw input folders and seeds test CSVs.

SEED_PATIENTS   = True
SEED_ENCOUNTERS = True
REPO_BASE       = "/Workspace/Users/matthew.databrickslab2@outlook.com/kardiaflow/data/raw"

RAW_PATHS = {
    "patients"  : "dbfs:/kardia/raw/patients/",
    "encounters": "dbfs:/kardia/raw/encounters/"
}

SEED_FILES = {
    "patients"  : f"file:{REPO_BASE}/ehr/patients_10.csv",
    "encounters": f"file:{REPO_BASE}/ehr/encounters_10.csv"
}

# Create folders if missing
for path in [
    *RAW_PATHS.values(),
    "dbfs:/kardia/_schemas/",
    "dbfs:/kardia/_checkpoints/"
]:
    dbutils.fs.mkdirs(path)

def safe_copy(src, dst):
    try:
        dbutils.fs.cp(src, dst, recurse=True)
        print(f"Copied {src} → {dst}")
    except Exception as e:
        print(f"Skipped {src}: {e}")

if SEED_PATIENTS:
    safe_copy(SEED_FILES["patients"],   RAW_PATHS["patients"])
if SEED_ENCOUNTERS:
    safe_copy(SEED_FILES["encounters"], RAW_PATHS["encounters"])

for name, path in RAW_PATHS.items():
    count = len(dbutils.fs.ls(path))
    print(f"{name.capitalize()} files: {count}")

print("Environment bootstrap complete")

# (DEMO ONLY) Minimize shuffle overhead for small test datasets
spark.conf.set("spark.sql.shuffle.partitions", "1")